In [1]:
import pandas as pd
from google import genai
from google.genai import types

In [2]:
english_definitions = pd.read_parquet("data/english_definitions.parquet")

In [3]:
english_definitions.head()

,index,characters,pinyin,definition
0,2,八,bā,eight; 8
1,3,爸爸,bàba,"(informal) father; CL:個|个[gè],位[wèi]"
2,4,吧,ba,(modal particle indicating polite suggestion);...
3,5,白,bái,white; snowy; pure; bright; empty; blank; plai...
4,6,百,bǎi,hundred; numerous; all kinds of; surname Bai


In [6]:
from gemini import translate_definitions_spanish

# send batches to gemini
n = 20  
batchs_ready = []
number_batch = 1

for start in range(0, len(english_definitions), n):
    batch = english_definitions.iloc[start : start + n]
    fixed_batched = translate_definitions_spanish(batch[:20].to_dict('records'))

    batchs_ready.append(fixed_batched)

    print("Finished batch: ", number_batch)
    number_batch+=1

Finished batch:  1
Finished batch:  2
Finished batch:  3
Finished batch:  4
Finished batch:  5
Finished batch:  6
Finished batch:  7
Finished batch:  8
Finished batch:  9
Finished batch:  10
Finished batch:  11
Finished batch:  12
Finished batch:  13
Finished batch:  14
Finished batch:  15
Finished batch:  16


In [21]:
spanish_definitions = pd.DataFrame([item for list_item in batchs_ready for item in list_item ])

chinese_spanish_vocabulary  = english_definitions.drop(columns='definition', axis=1).merge(spanish_definitions, how="left", left_on="index" , right_on="character_ID").drop(columns="character_ID")

# Save the data

In [22]:
chinese_spanish_vocabulary.to_parquet('data/chinese_spanish_vocabulary.parquet')